# Live Data Reduction

This page shows the rough idea of how the live data reduction can look like in jupyter notebook.

In [ ]:
import sys

sys.path.append('../../')  # To use ``docs`` as a package.
sys.path.append('../../tests/helpers/')  # To use ``tests`` as a package.

In [ ]:
%matplotlib widget
import scipp as sc

sc.get_logger().addHandler(sc.logging.make_widget_handler())
sc.display_logs()

In [ ]:
from tests.prototypes.prototype_mini import mini_prototype_factory, BeamlimeLogger, Prototype, PlotContainer, multiple_constant_providers
from tests.prototypes.parameters import EventRate, NumPixels, NumFrames
import logging

prototype_factory = mini_prototype_factory()
with multiple_constant_providers(
    prototype_factory,
    {
        BeamlimeLogger: sc.get_logger(),
        # Increase these numbers to make it slower.
        EventRate: EventRate(10_000),
        NumPixels: NumPixels(10_000),
        NumFrames: NumFrames(140)  
    }
):
    prototype_factory[BeamlimeLogger].setLevel(logging.DEBUG)
    prototype_factory[Prototype].run()


In [ ]:
shown = False
tried = 0
while not shown and (tried < 300):
    try:
        tried += 1
        assert prototype_factory[PlotContainer].histogram is not None
        shown = True
    except (AttributeError, AssertionError):
        ...

prototype_factory[PlotContainer].histogram